In [48]:
import pandas as pd

In [103]:
df_genderize = pd.read_csv("crew_genderize_data.csv", dtype={'name_id': str})

In [104]:
df_unique_names = pd.read_csv("parsed_names_all_unique_crew_for_inference.csv", dtype={"person_id": str})

In [105]:
df_gm = df_unique_names.merge(df_genderize[['gender', 'count', 'probability', 'name_id']], left_on="person_id", right_on="name_id")

In [106]:
df_gm['gender'].isnull().sum()

4789

In [107]:
df_gm = df_gm.drop(columns=['person_id'])

In [108]:
df_gm

,movie_id,file,role,name,long_canonical_name,headshot,name_parsed,gender,count,probability,name_id
0,2113683,4513_I_Know_That_Voice,cast,Devon Bowman,"Bowman, Devon",NaN,Devon Bowman,male,3308,0.79,1329505
1,2113683,4513_I_Know_That_Voice,cast,Morgan Gerhard,"Gerhard, Morgan",NaN,Morgan Gerhard,male,18372,0.79,1439900
2,2113683,4513_I_Know_That_Voice,cast,Gerald Burke,"Burke, Gerald",NaN,Gerald Burke,male,115901,1.00,2825311
3,2113683,4513_I_Know_That_Voice,cast,Kelly Ann Foley,"Foley, Kelly Ann",NaN,Kelly Ann Foley,female,62441,0.89,0284182
4,2113683,4513_I_Know_That_Voice,cast,Tom Lawless,"Lawless, Tom",NaN,Tom Lawless,male,698532,1.00,4905404
...,...,...,...,...,...,...,...,...,...,...,...
319320,3726322,2580_It_Never_Sleeps,editorial department,Mdhamiri Á Nkemi,"Nkemi, Mdhamiri Á",NaN,Mdhamiri Nkemi,NaN,0,0.00,4995023
319321,3726322,2580_It_Never_Sleeps,miscellaneous crew,Jon Anderson,"Anderson, Jon",NaN,Jon Anderson,male,249969,1.00,9196204
319322,3726322,2580_It_Never_Sleeps,miscellaneous crew,Harry Gadd,"Gadd, Harry",NaN,Harry Gadd,male,177229,1.00,6538683
319323,3726322,2580_It_Never_Sleeps,miscellaneous crew,Patricia Muckle,"Muckle, Patricia",NaN,Patricia Muckle,female,809021,1.00,6538684


In [109]:
df_cast = pd.read_csv("all_people_gender_merged.csv", dtype={'person_id': str})

In [110]:
df_cast['gender_named'] = df_cast['gender_named'].map({"F": "female", "M": "male"})

In [111]:
df_cast = df_cast[~df_cast['person'].duplicated()]

In [112]:
df_people = pd.read_csv("../support_crew/manual_crew/movies_support_crew_with_manual.csv", dtype=str)

In [113]:
df_all_gm = df_people.merge(df_cast[['person', 'gender_named']], 
                            left_on = "name", right_on='person', how='left')
df_all_gm = df_all_gm.rename(columns={"gender_named": "gender"})

In [114]:
df_all_gm.shape

(644969, 9)

In [115]:
(~df_all_gm['gender'].isnull()).sum()

91661

In [116]:
df_all_gm_with_crew = df_all_gm.merge(df_gm[['name', 'gender']],
                                      on="name", how="left")

In [117]:
df_all_gm_with_crew['gender'] = df_all_gm_with_crew['gender_x'].combine_first(df_all_gm_with_crew['gender_y'])

In [118]:
df_all_gm_with_crew = df_all_gm_with_crew.drop(columns=['gender_x', 'gender_y'])

In [119]:
df_all_gm_with_crew['gender'].isnull().sum()

7363

In [120]:
df_all_gm_with_crew.shape, df_people.shape

((644969, 9), (644969, 7))

In [121]:
df_all_gm_with_crew.to_csv("all_people_with_manual_gender_merged.csv", index=False)